## Load Dataset

In [1]:
datasets_root = 'datasets'
dataset_id = 'FUNSD_polygon_augmented'

In [2]:
from datasets import Dataset
from pathlib import Path
import json
from transformers import LayoutLMv3FeatureExtractor, AutoTokenizer, LayoutLMv3Processor
from datasets import Features, Sequence, ClassLabel, Value, Array2D
from PIL import Image
from functools import partial

### Load dataset
Generate a Dataset object from a generator function that contains the paths of the images and annotations.

In [3]:
def generate_samples(folder):
    images_path = Path(folder) / 'images'
    annotations_path = Path(folder) / 'annotations'
    
    images = list(images_path.glob('*.png'))
    
    for image_path in images:
        image_id = image_path.stem
        annotation_path = annotations_path / f'{image_id}.json'
        yield {
            'image_path': str(image_path),
            'annotation_path': str(annotation_path)
        }

def get_features_type():
    return Features({
        "image_path": Value("string"),
        "annotation_path": Value("string")
    })

def get_dataset_folder(split='train'):
    if split == 'train':
        path = Path(datasets_root) / dataset_id / 'dataset' / 'training_data'
    elif split == 'test':
        path = Path(datasets_root) / dataset_id / 'dataset' / 'testing_data'
    else:
        raise ValueError('split must be either "train" or "test"')
    
    return path



train_folder = get_dataset_folder()
train_dataset = Dataset.from_generator(partial(generate_samples, train_folder), features=get_features_type())
test_folder = get_dataset_folder('test')
test_dataset = Dataset.from_generator(partial(generate_samples, test_folder), features=get_features_type())

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
def get_unique_labels(folder):
    annotations_path = Path(folder) / 'annotations'
    unique_labels = set()

    for annotation_file in annotations_path.glob('*.json'):
        with open(annotation_file, 'r') as f:
            annotations = json.load(f)['form']
            for annotation in annotations:
                unique_labels.add(annotation['label'])

    return sorted(list(unique_labels))

labels = get_unique_labels(train_folder)
label_to_id = {label: idx for idx, label in enumerate(labels)}
id_to_label = {idx: label for idx, label in enumerate(labels)}

### Preprocess dataset
Load the images and annotations, and encode them using the LayoutLMv3 processor.

In [5]:
# Initialize processor
model_id = "SCUT-DLVCLab/lilt-roberta-en-base"
feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=False)
tokenizer = AutoTokenizer.from_pretrained(model_id)
processor = LayoutLMv3Processor(feature_extractor, tokenizer)

# Custom features
features = Features(
    {
        "input_ids": Sequence(feature=Value(dtype="int64")),
        "attention_mask": Sequence(feature=Value(dtype="int64")),
        "bbox": Array2D(dtype="int64", shape=(512, 8)),
        "labels": Sequence(ClassLabel(names=labels)),
    }
)

# Preprocess function
def process(sample, processor=None):
    # Assume sample["annotation_path"] is the path to the JSON file containing your annotations
    with open(sample["annotation_path"], "r") as f:
        annotations = json.load(f)['form']

    # Extract tokens and bounding boxes from annotations
    tokens = [ann["text"] for ann in annotations]
    bboxes = [ann["polygon"] for ann in annotations]  # Modify this if your "polygon" is not actually a bbox

    # Load image
    image = Image.open(sample["image_path"]).convert("RGB")
    
    # Convert string labels to integers using the mapping
    word_labels = [label_to_id[ann["label"]] for ann in annotations]

    # Encoding
    encoding = processor(
        image,
        tokens,
        boxes=bboxes,
        word_labels=word_labels,
        padding="max_length",
        truncation=True,
    )
    del encoding["pixel_values"]
    return encoding

# Process your dataset (replace `dataset` with the actual dataset object)
train_dataset = train_dataset.map(
    partial(process, processor=processor),
    remove_columns=["image_path", "annotation_path"],
    features=features,
).with_format("torch")
test_dataset = test_dataset.map(
    partial(process, processor=processor),
    remove_columns=["image_path", "annotation_path"],
    features=features,
).with_format("torch")

S:\Projects\polygon-document-comprehension\venv\lib\site-packages\transformers\models\layoutlmv3\feature_extraction_layoutlmv3.py:30: FutureWarning: The class LayoutLMv3FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use LayoutLMv3ImageProcessor instead.
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [6]:
from datasets import DatasetDict

dataset = DatasetDict({'train': train_dataset, 'test': test_dataset})
dataset.save_to_disk(Path(datasets_root) / dataset_id / 'huggingface_dataset')

Saving the dataset (0/2 shards):   0%|          | 0/14900 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]